# Load Lists

In [1]:
import pandas as pd

df = pd.read_csv('../data/mergedAttributeList.csv')
edge = pd.read_csv('../data/mergedEdgeList.csv')

# Check If Subreddit Exists

In [ ]:
import sys
import prawcore
import importlib
sys.path.append('../src/')

import reddit
reddit = importlib.reload(reddit)

import network
network = importlib.reload(network)

def subreddit_exists(reddit_name):
    print('starting')
    try:
        subreddit = reddit.reddit_api_wrapper.subreddit(reddit_name)
        print(subreddit.name)
        return True
    except prawcore.exceptions.RequestException:
        print('recursing')
        return subreddit_exists(reddit_name)
    except KeyboardInterrupt:
        print('foobar')
        raise
    except:
        e = sys.exc_info()[0]
        print(e)
        return e

df['exists'] = df.reddit_name.map(subreddit_exists)

# Remove Non-Existent Subreddits

In [14]:
# remove every marked subreddit on attribute list
df[df['exists'] == True][['reddit_name', 'desc', 'created_date', 'subs']].to_csv('../data/verifiedAttributeList.csv')
# remove every record in edge list with that subreddit as source or reddit
verified_reddits = df[df['exists'] == True].reddit_name.unique()
edge[edge.source.isin(verified_reddits) & edge.target.isin(verified_reddits)].to_csv('../data/verifiedEdgeList.csv')